# Battery Data Tool - Standalone Notebook

모든 필요한 함수가 노트북 내에 포함된 자급자족형 분석 도구

**데이터 경로**: `c:\Users\Ryu\Python_project\data\battery251027\Rawdata`

## 1. 라이브러리 Import

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.ndimage import gaussian_filter1d

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (14, 8)

# 기본 경로
RAWDATA_PATH = r'c:\Users\Ryu\Python_project\data\battery251027\Rawdata'

# 그래프 색상
COLORS = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', 
          '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

print("✅ 라이브러리 로드 완료")

## 2. 핵심 유틸리티 함수

In [ ]:
def check_cycler(raw_file_path):
    """PNE와 TOYO 충방전기 구분 (Pattern 폴더 유무로 판단)"""
    return os.path.isdir(os.path.join(raw_file_path, "Pattern"))

def progress(folder_cnt, folder_max, chnl_cnt, chnl_max, cyc_cnt, cyc_max):
    """진행률 계산"""
    if cyc_max == 0:
        return 0
    return ((folder_cnt + (chnl_cnt + cyc_cnt/cyc_max - 1)/chnl_max - 1)/folder_max) * 100

def get_channels(test_path):
    """채널 폴더 목록 반환"""
    return [f for f in os.listdir(test_path) 
            if os.path.isdir(os.path.join(test_path, f)) and 'Ch' in f]

print("✅ 유틸리티 함수 정의 완료")

## 3. PNE 사이클 데이터 로더 (간소화 버전)

In [ ]:
def load_pne_cycle_simple(channel_path, mincapacity=5000):
    """
    PNE 사이클 데이터 로드 (간소화 버전)
    
    Returns:
        capacity: float
        df: DataFrame with columns ['방전용량', '충전용량', '충방효율', 'Rest End', 'dcir', '방전Energy']
    """
    try:
        # .cyc 파일 찾기
        cyc_files = glob.glob(os.path.join(channel_path, '*.cyc'))
        if not cyc_files:
            return None, None
        
        cyc_file = cyc_files[0]
        
        # 데이터 로드 (PNE 형식)
        df_raw = pd.read_csv(
            cyc_file,
            sep='\t',
            encoding='cp949',
            skiprows=1,
            on_bad_lines='skip',
            low_memory=False
        )
        
        # 필요한 컬럼만 추출
        cycle_data = pd.DataFrame()
        
        if len(df_raw.columns) > 11:
            # Step 2인 방전 데이터만 추출
            discharge_mask = (df_raw.iloc[:, 2] == 2) & (df_raw.iloc[:, 6] == 65)
            discharge_data = df_raw[discharge_mask]
            
            if len(discharge_data) > 0:
                # 사이클별로 그룹화
                grouped = discharge_data.groupby(discharge_data.iloc[:, 27])  # 27: TotalCycle
                
                cycle_numbers = []
                dchg_cap = []
                chg_cap = []
                efficiency = []
                rest_voltage = []
                energy = []
                
                for cycle_num, group in grouped:
                    if len(group) > 0:
                        cycle_numbers.append(int(cycle_num))
                        dchg_cap.append(group.iloc[-1, 11])  # 11: DchgCapacity
                        chg_cap.append(group.iloc[-1, 10])   # 10: ChgCapacity
                        
                        # 효율 계산
                        if group.iloc[-1, 10] > 0:
                            eff = (group.iloc[-1, 11] / group.iloc[-1, 10]) * 100
                        else:
                            eff = 0
                        efficiency.append(eff)
                        
                        rest_voltage.append(group.iloc[-1, 8] / 1000)  # 8: Voltage(mV)
                        energy.append(group.iloc[-1, 15])  # 15: DchgWattHour
                
                cycle_data = pd.DataFrame({
                    '방전용량': dchg_cap,
                    '충전용량': chg_cap,
                    '충방효율': efficiency,
                    'Rest End': rest_voltage,
                    '방전Energy': energy
                }, index=cycle_numbers)
                
                # 용량 계산 (첫 사이클 기준)
                if len(dchg_cap) > 0 and dchg_cap[0] > 0:
                    capacity = dchg_cap[0]
                else:
                    capacity = mincapacity
                
                return capacity, cycle_data
        
        return mincapacity, cycle_data
        
    except Exception as e:
        print(f"Error loading PNE data: {e}")
        return None, None

print("✅ PNE 로더 정의 완료")

## 4. TOYO 사이클 데이터 로더 (간소화 버전)

In [ ]:
def load_toyo_cycle_simple(channel_path, mincapacity=5000):
    """
    TOYO 사이클 데이터 로드 (간소화 버전)
    
    Returns:
        capacity: float
        df: DataFrame with columns ['방전용량', '충전용량', '충방효율', 'Rest End', '방전Energy']
    """
    try:
        # .cyc 파일 찾기
        cyc_files = glob.glob(os.path.join(channel_path, '*.cyc'))
        if not cyc_files:
            return None, None
        
        cyc_file = cyc_files[0]
        
        # 데이터 로드 (TOYO 형식)
        df_raw = pd.read_csv(
            cyc_file,
            sep='\t',
            encoding='cp949',
            skiprows=1,
            on_bad_lines='skip',
            low_memory=False
        )
        
        cycle_data = pd.DataFrame()
        
        if len(df_raw.columns) > 10:
            # TOYO 형식: 사이클별 데이터 추출
            cycle_numbers = df_raw.iloc[:, 0].values  # Cycle number
            dchg_cap = df_raw.iloc[:, 5].values       # Discharge capacity
            chg_cap = df_raw.iloc[:, 4].values        # Charge capacity
            rest_voltage = df_raw.iloc[:, 8].values   # Rest voltage
            energy = df_raw.iloc[:, 7].values         # Discharge energy
            
            # 효율 계산
            efficiency = np.where(chg_cap > 0, (dchg_cap / chg_cap) * 100, 0)
            
            cycle_data = pd.DataFrame({
                '방전용량': dchg_cap,
                '충전용량': chg_cap,
                '충방효율': efficiency,
                'Rest End': rest_voltage,
                '방전Energy': energy
            }, index=cycle_numbers.astype(int))
            
            # 용량 계산
            if len(dchg_cap) > 0 and dchg_cap[0] > 0:
                capacity = dchg_cap[0]
            else:
                capacity = mincapacity
            
            return capacity, cycle_data
        
        return mincapacity, cycle_data
        
    except Exception as e:
        print(f"Error loading TOYO data: {e}")
        return None, None

print("✅ TOYO 로더 정의 완료")

## 5. 통합 사이클 데이터 로더

In [ ]:
def load_cycle_data(channel_path, mincapacity=5000):
    """
    PNE/TOYO 자동 감지하여 사이클 데이터 로드
    
    Returns:
        capacity: float
        df: DataFrame
        cycler_type: str ('PNE' or 'TOYO')
    """
    parent_path = os.path.dirname(channel_path)
    is_pne = check_cycler(parent_path)
    
    if is_pne:
        capacity, df = load_pne_cycle_simple(channel_path, mincapacity)
        return capacity, df, 'PNE'
    else:
        capacity, df = load_toyo_cycle_simple(channel_path, mincapacity)
        return capacity, df, 'TOYO'

print("✅ 통합 로더 정의 완료")

## 6. 사용 가능한 테스트 폴더 탐색

In [ ]:
# 사용 가능한 테스트 폴더 목록
test_folders = [f for f in os.listdir(RAWDATA_PATH) 
                if os.path.isdir(os.path.join(RAWDATA_PATH, f))]

print("📁 사용 가능한 테스트 폴더:")
print("=" * 100)

for idx, folder in enumerate(test_folders, 1):
    folder_path = os.path.join(RAWDATA_PATH, folder)
    is_pne = check_cycler(folder_path)
    cycler_type = "PNE" if is_pne else "TOYO"
    channels = get_channels(folder_path)
    
    print(f"{idx:2d}. [{cycler_type:4s}] {folder[:80]} ({len(channels)} channels)")

print("=" * 100)

## 7. 예제 1: 개별 사이클 분석 (Individual Cycle Analysis)

In [ ]:
# ===== 설정 =====
TEST_FOLDER = 'A1_MP1_4500mAh_T23_1'  # 분석할 테스트 폴더명
MIN_CAPACITY = 4500                    # 최소 용량 (mAh)
MAX_CHANNELS = 2                       # 분석할 최대 채널 수

# ===== 분석 실행 =====
test_path = os.path.join(RAWDATA_PATH, TEST_FOLDER)

if not os.path.exists(test_path):
    print(f"❌ 폴더를 찾을 수 없습니다: {TEST_FOLDER}")
else:
    print(f"\n{'='*100}")
    print(f"📊 개별 사이클 분석: {TEST_FOLDER}")
    print(f"{'='*100}\n")
    
    channels = get_channels(test_path)
    print(f"발견된 채널: {len(channels)}개\n")
    
    for ch_idx, channel in enumerate(channels[:MAX_CHANNELS]):
        channel_path = os.path.join(test_path, channel)
        print(f"\n[{ch_idx+1}/{min(MAX_CHANNELS, len(channels))}] {channel} 분석 중...")
        
        # 데이터 로드
        capacity, df, cycler_type = load_cycle_data(channel_path, MIN_CAPACITY)
        
        if df is None or len(df) == 0:
            print(f"  ⚠️  데이터 없음")
            continue
        
        print(f"  ✅ {cycler_type} 데이터 로드: {len(df)} cycles, Capacity: {capacity:.1f} mAh")
        
        # 6-panel 그래프 생성
        fig, axes = plt.subplots(2, 3, figsize=(16, 9))
        cycles = df.index.values
        
        # Panel 1: 방전 용량
        if '방전용량' in df.columns:
            axes[0, 0].plot(cycles, df['방전용량'], 'o-', color=COLORS[0], markersize=4, linewidth=2)
            axes[0, 0].set_title('방전 용량', fontsize=12, fontweight='bold')
            axes[0, 0].set_xlabel('Cycle')
            axes[0, 0].set_ylabel('Capacity (mAh)')
            axes[0, 0].grid(True, alpha=0.3)
        
        # Panel 2: 충전 용량
        if '충전용량' in df.columns:
            axes[0, 1].plot(cycles, df['충전용량'], 'o-', color=COLORS[1], markersize=4, linewidth=2)
            axes[0, 1].set_title('충전 용량', fontsize=12, fontweight='bold')
            axes[0, 1].set_xlabel('Cycle')
            axes[0, 1].set_ylabel('Capacity (mAh)')
            axes[0, 1].grid(True, alpha=0.3)
        
        # Panel 3: 충방전 효율
        if '충방효율' in df.columns:
            axes[0, 2].plot(cycles, df['충방효율'], 'o-', color=COLORS[2], markersize=4, linewidth=2)
            axes[0, 2].set_title('충방전 효율', fontsize=12, fontweight='bold')
            axes[0, 2].set_xlabel('Cycle')
            axes[0, 2].set_ylabel('Efficiency (%)')
            axes[0, 2].set_ylim([95, 105])
            axes[0, 2].grid(True, alpha=0.3)
        
        # Panel 4: Rest 전압
        if 'Rest End' in df.columns:
            axes[1, 0].plot(cycles, df['Rest End'], 'o-', color=COLORS[3], markersize=4, linewidth=2)
            axes[1, 0].set_title('Rest 전압', fontsize=12, fontweight='bold')
            axes[1, 0].set_xlabel('Cycle')
            axes[1, 0].set_ylabel('Voltage (V)')
            axes[1, 0].grid(True, alpha=0.3)
        
        # Panel 5: 방전 에너지
        if '방전Energy' in df.columns:
            axes[1, 1].plot(cycles, df['방전Energy'], 'o-', color=COLORS[4], markersize=4, linewidth=2)
            axes[1, 1].set_title('방전 에너지', fontsize=12, fontweight='bold')
            axes[1, 1].set_xlabel('Cycle')
            axes[1, 1].set_ylabel('Energy (Wh)')
            axes[1, 1].grid(True, alpha=0.3)
        
        # Panel 6: 용량 유지율
        if '방전용량' in df.columns and len(df) > 0:
            initial_cap = df['방전용량'].iloc[0]
            retention = (df['방전용량'] / initial_cap) * 100
            axes[1, 2].plot(cycles, retention, 'o-', color=COLORS[5], markersize=4, linewidth=2)
            axes[1, 2].set_title('용량 유지율', fontsize=12, fontweight='bold')
            axes[1, 2].set_xlabel('Cycle')
            axes[1, 2].set_ylabel('Retention (%)')
            axes[1, 2].set_ylim([90, 105])
            axes[1, 2].grid(True, alpha=0.3)
        
        plt.suptitle(f'{TEST_FOLDER} - {channel} [{cycler_type}]', 
                    fontsize=16, fontweight='bold', y=0.995)
        plt.tight_layout()
        plt.show()
        
        # 통계 출력
        print(f"\n  📊 통계 정보:")
        print(f"     초기 용량: {df['방전용량'].iloc[0]:.1f} mAh")
        print(f"     최종 용량: {df['방전용량'].iloc[-1]:.1f} mAh")
        print(f"     용량 유지율: {(df['방전용량'].iloc[-1]/df['방전용량'].iloc[0]*100):.2f}%")
        print(f"     평균 효율: {df['충방효율'].mean():.2f}%")

print("\n" + "="*100)

## 8. 예제 2: 여러 테스트 비교 분석

In [ ]:
# ===== 설정 =====
COMPARE_FOLDERS = [
    'A1_MP1_4500mAh_T23_1',
    'A1_MP1_4500mAh_T23_2',
    'A1_MP1_4500mAh_T23_3'
]
MIN_CAPACITY = 4500

# ===== 비교 분석 =====
print(f"\n{'='*100}")
print(f"📊 여러 테스트 비교 분석")
print(f"{'='*100}\n")

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

for idx, folder in enumerate(COMPARE_FOLDERS):
    test_path = os.path.join(RAWDATA_PATH, folder)
    
    if not os.path.exists(test_path):
        print(f"⚠️  폴더 없음: {folder}")
        continue
    
    channels = get_channels(test_path)
    if len(channels) == 0:
        continue
    
    # 첫 번째 채널 데이터 로드
    channel_path = os.path.join(test_path, channels[0])
    capacity, df, cycler_type = load_cycle_data(channel_path, MIN_CAPACITY)
    
    if df is None or len(df) == 0:
        continue
    
    print(f"✅ {folder}: {len(df)} cycles, {capacity:.1f} mAh [{cycler_type}]")
    
    cycles = df.index.values
    color = COLORS[idx % len(COLORS)]
    label = f"Test {idx+1}"
    
    # 4개 그래프에 모두 플롯
    if '방전용량' in df.columns:
        axes[0, 0].plot(cycles, df['방전용량'], 'o-', color=color, label=label, markersize=3, alpha=0.7)
    
    if '충방효율' in df.columns:
        axes[0, 1].plot(cycles, df['충방효율'], 'o-', color=color, label=label, markersize=3, alpha=0.7)
    
    if 'Rest End' in df.columns:
        axes[1, 0].plot(cycles, df['Rest End'], 'o-', color=color, label=label, markersize=3, alpha=0.7)
    
    if '방전용량' in df.columns and len(df) > 0:
        retention = (df['방전용량'] / df['방전용량'].iloc[0]) * 100
        axes[1, 1].plot(cycles, retention, 'o-', color=color, label=label, markersize=3, alpha=0.7)

# 그래프 설정
axes[0, 0].set_title('방전 용량 비교', fontweight='bold')
axes[0, 0].set_xlabel('Cycle')
axes[0, 0].set_ylabel('Capacity (mAh)')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].set_title('충방전 효율 비교', fontweight='bold')
axes[0, 1].set_xlabel('Cycle')
axes[0, 1].set_ylabel('Efficiency (%)')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

axes[1, 0].set_title('Rest 전압 비교', fontweight='bold')
axes[1, 0].set_xlabel('Cycle')
axes[1, 0].set_ylabel('Voltage (V)')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].set_title('용량 유지율 비교', fontweight='bold')
axes[1, 1].set_xlabel('Cycle')
axes[1, 1].set_ylabel('Retention (%)')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.suptitle('테스트 간 비교 분석', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n" + "="*100)

## 9. 예제 3: Excel 데이터 내보내기

In [ ]:
# ===== 설정 =====
EXPORT_FOLDER = 'A1_MP1_4500mAh_T23_1'
OUTPUT_FILE = 'cycle_analysis_output.xlsx'

# ===== 데이터 수집 및 내보내기 =====
test_path = os.path.join(RAWDATA_PATH, EXPORT_FOLDER)

if os.path.exists(test_path):
    print(f"\n📄 Excel 내보내기: {EXPORT_FOLDER}")
    
    channels = get_channels(test_path)
    output_path = os.path.join(os.path.dirname(RAWDATA_PATH), OUTPUT_FILE)
    
    with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
        for ch_idx, channel in enumerate(channels[:5]):  # 최대 5개 채널
            channel_path = os.path.join(test_path, channel)
            capacity, df, cycler_type = load_cycle_data(channel_path, 4500)
            
            if df is not None and len(df) > 0:
                sheet_name = f"Ch{ch_idx+1}"
                df.to_excel(writer, sheet_name=sheet_name, index_label='Cycle')
                print(f"  ✅ {channel} → Sheet '{sheet_name}' ({len(df)} rows)")
    
    print(f"\n💾 저장 완료: {output_path}")
else:
    print(f"❌ 폴더를 찾을 수 없습니다: {EXPORT_FOLDER}")

## 10. 사용 가능한 분석 함수 요약

In [ ]:
print("""
╔═══════════════════════════════════════════════════════════════════════════════╗
║                   Battery Data Tool - Standalone Notebook                     ║
╚═══════════════════════════════════════════════════════════════════════════════╝

✅ 포함된 기능:

1. check_cycler(path)
   - PNE/TOYO 충방전기 자동 감지

2. load_pne_cycle_simple(channel_path, mincapacity)
   - PNE 사이클 데이터 로드

3. load_toyo_cycle_simple(channel_path, mincapacity)
   - TOYO 사이클 데이터 로드

4. load_cycle_data(channel_path, mincapacity)
   - 통합 데이터 로더 (자동 감지)

5. get_channels(test_path)
   - 채널 폴더 목록 반환

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

📊 예제 노트북:

- 예제 1: 개별 사이클 분석 (6-panel 그래프)
- 예제 2: 여러 테스트 비교 분석 (4-panel 비교)
- 예제 3: Excel 데이터 내보내기

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

📁 데이터 경로: c:\\Users\\Ryu\\Python_project\\data\\battery251027\\Rawdata

🔧 외부 의존성: 없음 (모든 함수가 노트북 내에 포함됨)

╚═══════════════════════════════════════════════════════════════════════════════╝
""")